In [78]:
import spotipy.util as util

username = 'sanoseyni5'
client_id ='302125874eea4e4dbab4f18a03d615e7'
client_secret = 'cfc7197b06ca46238546455d2d67950f'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [2]:
print(token)

BQCrUu5opDR0MPBUJ7P6s691Nfv_Tx-5zE1wQr4a85nxUbP1OvNACEl_zwCl6dhoIxNyN5KUdGj9bEDu8EB3Lu48bL3GO5-6XoJHdFqQZd3rMkEK8NiyFmTGfiQwr7I1NrFs6c4MGMeagOgO9_4y1uw


In [23]:
import spotipy

In [74]:
import requests
def get_id(track_name: str, token: str) -> str:
    headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer ' + token,
    }
    params = [
    ('q', track_name),
    ('type', 'track'),
    ]
    try:
        response = requests.get('https://api.spotify.com/v1/search', 
                    headers = headers, params = params, timeout = 5)
        json = response.json()
        first_result = json['tracks']['items'][0]
        track_id = first_result['id']
        return track_id

    except:
        return None

In [75]:
lucy_id = get_id('Et un jour une femme Florent Pagny', token)
print(lucy_id)

2Fo8TPJGFZZ7I9U2RGeZWu


In [21]:
def get_features(track_id: str, token: str) -> dict:
    sp = spotipy.Spotify(auth=token)
    try:
        features = sp.audio_features([track_id])
        return features[0]
    except:
        return None

In [26]:
lucy_features = get_features(lucy_id, token)
print(lucy_features)

{'danceability': 0.634, 'energy': 0.387, 'key': 9, 'loudness': -10.522, 'mode': 0, 'speechiness': 0.0282, 'acousticness': 0.217, 'instrumentalness': 0.00102, 'liveness': 0.0591, 'valence': 0.16, 'tempo': 135.687, 'type': 'audio_features', 'id': '2Fo8TPJGFZZ7I9U2RGeZWu', 'uri': 'spotify:track:2Fo8TPJGFZZ7I9U2RGeZWu', 'track_href': 'https://api.spotify.com/v1/tracks/2Fo8TPJGFZZ7I9U2RGeZWu', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2Fo8TPJGFZZ7I9U2RGeZWu', 'duration_ms': 319493, 'time_signature': 4}


In [34]:
import ast
from typing import List
from os import listdir
def get_streamings(path: str = 'C:/Users/sanos/Downloads/my_spotify_data/MyData') -> List[dict]:
    
    files = ['C:/Users/sanos/Downloads/my_spotify_data/MyData/' + x for x in listdir(path)
             if x.split('.')[0][:-1] == 'StreamingHistory']
    
    all_streamings = []
    
    for file in files: 
        with open(file, 'r', encoding='UTF-8') as f:
            new_streamings = ast.literal_eval(f.read())
            all_streamings += [streaming for streaming 
                               in new_streamings]
    return all_streamings

In [79]:
streamings = get_streamings()
unique_tracks = list(set([streaming['trackName']+' '+streaming['artistName']
                for streaming in streamings]))

all_features = {}
for track in unique_tracks:
    track_id = get_id(track, token)
    features = get_features(track_id, token)
    if features:
        all_features[track] = features
        
with_features = []
for track_name, features in all_features.items():
    with_features.append({'name': track_name, **features})

In [80]:
import pandas as pd
df = pd.DataFrame(with_features)
df.to_csv('streaming_history.csv')

In [81]:
df.head()

,name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Best Day Of My Life American Authors,0.675,0.894,2,-2.442,1,0.0355,0.0599,0.000452,0.0484,0.537,100.010,audio_features,0Dho96aJIZ5LJZXyPllbWA,spotify:track:0Dho96aJIZ5LJZXyPllbWA,https://api.spotify.com/v1/tracks/0Dho96aJIZ5L...,https://api.spotify.com/v1/audio-analysis/0Dho...,194187,4
1,Ça marche (feat. The Shin Sekaï) GIMS,0.780,0.848,0,-4.422,1,0.0642,0.0130,0.000000,0.3190,0.789,120.030,audio_features,7lg11NyWmYxjCv7RVr80ON,spotify:track:7lg11NyWmYxjCv7RVr80ON,https://api.spotify.com/v1/tracks/7lg11NyWmYxj...,https://api.spotify.com/v1/audio-analysis/7lg1...,221013,4
2,I Like The Way Timaya,0.802,0.790,0,-2.794,1,0.0710,0.0686,0.000018,0.0747,0.683,101.030,audio_features,1fDMlFjBdb1A2fPDBN5Wcg,spotify:track:1fDMlFjBdb1A2fPDBN5Wcg,https://api.spotify.com/v1/tracks/1fDMlFjBdb1A...,https://api.spotify.com/v1/audio-analysis/1fDM...,199933,4
3,Tu ne le vois pas (feat. Dadju) GIMS,0.751,0.620,10,-6.941,0,0.0783,0.0278,0.000000,0.1480,0.419,103.953,audio_features,2CIt98hwNBUGhhYSX6GQPU,spotify:track:2CIt98hwNBUGhhYSX6GQPU,https://api.spotify.com/v1/tracks/2CIt98hwNBUG...,https://api.spotify.com/v1/audio-analysis/2CIt...,261840,4
4,Bonita GIMS,0.666,0.586,8,-8.031,0,0.0584,0.4670,0.000003,0.0945,0.757,124.008,audio_features,3zisukp9wkrjfJ4vJNUk1n,spotify:track:3zisukp9wkrjfJ4vJNUk1n,https://api.spotify.com/v1/tracks/3zisukp9wkrj...,https://api.spotify.com/v1/audio-analysis/3zis...,213533,4


In [86]:
headers = {'Authorization': 'Bearer {token}'.format(token=token)}

In [90]:
dict_genre = {}

# convert track_uri column to an iterable list
track_uris = df['uri'].to_list()

# loop through track URIs and pull artist URI using the API,
# then use artist URI to pull genres associated with that artist
# store all these in a dictionary
for t_uri in track_uris:
    
    dict_genre[t_uri] = {'artist_uri': "", "genres":[]}
    
    r = requests.get('https://api.spotify.com/v1/' + 'tracks/' + t_uri, headers=headers)
    r = r.json()
    a_uri = r['artists'][0]['uri'].split(':')[2]
    dict_genre[t_uri]['artist_uri'] = a_uri
    
    s = requests.get('https://api.spotify.com/v1/' + 'artists/' + a_uri, headers=headers)
    s = s.json()
    dict_genre[t_uri]['genres'] = s['genres']

KeyError: 'artists'